In [55]:
import numpy as np
import pandas as pd

df = pd.read_csv('telecom.csv')

In [1]:
df.head() # Выводим 5 первых строк таблицы

In [15]:
df.shape # Посмотрим на размер данных. Размерность таблицы n х m

In [16]:
df.columns # Выведем названия столбцов

In [ ]:
df.info() # Общая информация по датафрейму и всем признакам

In [ ]:
# Изменить тип колонки можно с помощью метода astype. Применим этот метод к признаку Churn и переведём его в int64
df['churn'] = df['churn'].astype('int64')

In [5]:
df.info()

#### Показывает основные статистические характеристики данных по каждому числовому признаку (типы int64 и float64): число непропущенных значений, среднее, стандартное отклонение, диапазон, медиану, 0.25 и 0.75 квартили.

In [18]:
df.describe()

#### Чтобы посмотреть статистику по нечисловым признакам, нужно явно указать интересующие нас типы в параметре include.

In [ ]:
df.describe(include=['object', 'bool'])

#### Для категориальных (тип object) и булевых (тип bool) признаков можно воспользоваться методом value_counts. Посмотрим на распределение данных по нашей целевой переменной — Churn:

In [ ]:
df['churn'].value_counts()

#### Посмотрим на распределение пользователей по переменной Area code. Укажем значение параметра normalize=True, чтобы посмотреть не абсолютные частоты, а относительные.

In [ ]:
df['area code'].value_counts(normalize=True)

### Сортировка

#### DataFrame можно отсортировать по значению какого-нибудь из признаков. В нашем случае, например, по Total day charge (ascending=False для сортировки по убыванию):

In [ ]:
print(df.sort_values(by='total day charge', ascending=False).head())

#### ортировать можно и по группе столбцов:

In [ ]:
df.sort_values(by=['churn', 'total day charge'], ascending=[True, False]).head()

### Индексация и извлечение данных

DataFrame можно индексировать по-разному. В связи с этим рассмотрим различные способы индексации и извлечения нужных нам данных из датафрейма на примере простых вопросов. 

Для извлечения отдельного столбца можно использовать конструкцию вида DataFrame['Name']. Воспользуемся этим для ответа на вопрос: какова доля людей нелояльных пользователей в нашем датафрейме?

In [6]:
'Доля нелояльных клиентов', df['churn'].mean()

('Доля нелояльных клиентов', 0.14491449144914492)

Каковы средние значения числовых признаков среди нелояльных пользователей?

In [8]:
df[df['churn'] == 1].mean()

Сколько в среднем в течение дня разговаривают по телефону нелояльные пользователи

In [17]:
'Нелояльные разговаривают в день, мин', df[df['churn'] == 1]['total day minutes'].mean(), \
'Лояльные разговаривают в день, мин', df[df['churn'] == 0]['total day minutes'].mean()

Какова максимальная длина международных звонков среди лояльных пользователей (Churn == 0), не пользующихся услугой международного роуминга ('International plan' == 'No')?

In [18]:
df[(df['churn'] == 0) & (df['international plan'] == 'no')]['total intl minutes'].max()

18.899999999999999

Датафреймы можно индексировать как по названию столбца или строки, так и по порядковому номеру.
Для индексации по названию используется метод loc, по номеру — iloc.
В первом случае мы говорим «передай нам значения для id строк от 0 до 5 и для столбцов от State до Area code», а во втором — «передай нам значения первых пяти строк в первых трёх столбцах».

In [22]:
df.loc[0:5, 'state':'area code']
df.iloc[0:5, 0:3]

Если нам нужна первая или последняя строчка датафрейма, пользуемся конструкцией df[:1] или df[-1:]:

In [24]:
df[-1:]

### Применение функций к ячейкам, столбцам и строкам

Применение функции к каждому столбцу: apply

In [ ]:
df.apply(np.max)

#### Применение функции к каждой ячейке столбца: map

Например, метод map можно использовать для замены значений в колонке, передав ему в качестве аргумента словарь вида {old_value: new_value}:

In [26]:
d = {'no' : False, 'yes' : True}
df['international plan'] = df['international plan'].map(d)
df.head()

Аналогичную операцию можно провернуть с помощью метода replace:

In [ ]:
df = df.replace({'voice mail plan': 'd'})
df.head()

### Группировка данных

В общем случае группировка данных в Pandas выглядит следующим образом:

In [ ]:
df.groupby(by=grouping_columns)[columns_to_show].function()

К датафрейму применяется метод groupby, который разделяет данные по grouping_columns – признаку или набору признаков.

Выбираем нужные нам столбцы (columns_to_show). К полученным группам применяется функция или несколько функций.

#### Группирование данных в зависимости от значения признака Churn и вывод статистик по трём столбцам в каждой группе.

In [ ]:
columns_to_show = ['total day minutes', 'total eve minutes', 'total night minutes']
df.groupby(['churn'])[columns_to_show].describe(percentiles=[])

Сделаем то же самое, но немного по-другому, передав в agg список функций:

In [ ]:
columns_to_show = ['total day minutes', 'total eve minutes', 'total night minutes']
df.groupby(['churn'])[columns_to_show].agg([np.mean, np.std, np.min, np.max])

### Сводные таблицы

Допустим, мы хотим посмотреть, как наблюдения в нашей выборке распределены в контексте двух признаков — Churn и International plan. Для этого мы можем построить таблицу сопряженности, воспользовавшись методом crosstab:

In [31]:
pd.crosstab(df['churn'], df['international plan'])
pd.crosstab(df['churn'], df['voice mail plan'], normalize=True)

Мы видим, что большинство пользователей лояльны и при этом пользуются дополнительными услугами (международного роуминга / голосовой почты).

Давайте посмотрим среднее число дневных, вечерних и ночных звонков для разных Area code:

In [32]:
df.pivot_table(['total day calls', 'total eve calls', 'total night calls'], ['area code'], aggfunc='mean').head(10)

,total day calls,total eve calls,total night calls
area code,,,
408,100.496420,99.788783,99.039379
415,100.576435,100.503927,100.398187
510,100.097619,99.671429,100.601190


### Преобразование датафреймов

Мы хотим посчитать общее количество звонков для всех пользователей. Создадим объект total_calls типа Series и вставим его в датафрейм:

In [44]:
total_calls = df['total day calls'] + df['total eve calls'] + df['total night calls'] + df['total intl calls']
df.insert(loc=len(df.columns), column='total calls', value=total_calls)

loc - номер столбца, после которого нужно вставить данный Series 

мы указали len(df.columns), чтобы вставить его в самом конце

In [45]:
df.head()

Добавить столбец из имеющихся можно и проще, не создавая промежуточных Series:

In [47]:
df['total charge'] = df['total day charge'] + df['total eve charge'] + df['total night charge'] + df['total intl charge']
df.head()

Чтобы удалить столбцы или строки, воспользуйтесь методом drop, передавая в качестве аргумента нужные индексы и требуемое значение параметра axis (1, если удаляете столбцы, и ничего или 0, если удаляете строки): избавляемся от созданных только что столбцов

In [50]:
df = df.drop(['total charge', 'total calls'], axis=1)
df.drop([1, 2]).head() # а вот так можно удалить строчки

### Первые попытки прогнозирования оттока

Посмотрим, как отток связан с признаком "Подключение международного роуминга" (International plan).

Сделаем это с помощью сводной таблички crosstab, а также путем иллюстрации с Seaborn (как именно строить такие картинки и анализировать с их помощью графики – материал следующей статьи).

In [53]:
pd.crosstab(df['churn'], df['international plan'], margins=True)

international plan,no,yes,All
churn,,,
False,2664,186,2850
True,346,137,483
All,3010,323,3333


Видим, что когда роуминг подключен, доля оттока намного выше – интересное наблюдение! Возможно, большие и плохо контролируемые траты в роуминге очень конфликтогенны и приводят к недовольству клиентов телеком-оператора и, соответственно, к их оттоку.

Далее посмотрим на еще один важный признак – "Число обращений в сервисный центр" (Customer service calls). Также построим сводную таблицу и картинку.

In [58]:
pd.crosstab(df['churn'], df['customer service calls'], margins=True)

customer service calls,0,1,2,3,4,5,6,7,8,9,All
churn,,,,,,,,,,,
False,605,1059,672,385,90,26,8,4,1,0,2850
True,92,122,87,44,76,40,14,5,1,2,483
All,697,1181,759,429,166,66,22,9,2,2,3333


Может быть, по сводной табличке это не так хорошо видно (или скучно ползать взглядом по строчкам с цифрами), а вот картинка красноречиво свидетельствует о том, что доля оттока сильно возрастает начиная с 4 звонков в сервисный центр.

Добавим теперь в наш DataFrame бинарный признак — результат сравнения Customer service calls > 3. И еще раз посмотрим, как он связан с оттоком.

In [60]:
df['many_service_calls'] = (df['customer service calls'] > 3).astype('int')
pd.crosstab(df['many_service_calls'], df['churn'], margins=True)

churn,False,True,All
many_service_calls,,,
0,2721,345,3066
1,129,138,267
All,2850,483,3333


Объединим рассмотренные выше условия и построим сводную табличку для этого объединения и оттока.

In [61]:
pd.crosstab(df['many_service_calls'] & df['international plan'] , df['churn'])

churn,False,True
row_0,,
False,2721,345
True,129,138
